# Entrenamiento y exploración del dimensionamiento de los datos

In [12]:
import os
import torch

from utilit_exploracion             import get_exp_name, load_data, apply_autoreject, split_dataset
from sklearn.utils.class_weight     import compute_class_weight

In [2]:
cuda   = torch.cuda.is_available()  # check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if cuda:
    torch.backends.cudnn.enabled       = True
    torch.backends.cudnn.benchmark     = not True # args.deterministic
    torch.backends.cudnn.deterministic = True     # args.deterministic

In [3]:
# create savedir

dir_name  = get_exp_name('pc18_debug', 'stager_net', 'dsfd', 'autoreject') # 'no_denoising')
save_dir  = './runs/pc18_debug/'
save_path = os.path.join(save_dir, dir_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
# load, preprocess and window data

windows_dataset = load_data('pc18_debug', 30, 4)
if  True:                                                      #args.denoising == 'autoreject':
    windows_dataset = apply_autoreject(windows_dataset, 87, 4) # apply_autoreject(windows_dataset, args.seed, args.n_jobs)

In [6]:
# split into train, valid and test sets

available_classes = windows_dataset.get_metadata()['target'].unique()

train_set, valid_set, test_set = split_dataset(
                                                windows_dataset, 
                                                0.2,                     # args.valid_size, 
                                                0.2,                     # args.test_size,
                                                random_state_valid = 87, # =args.random_state_valid,
                                                random_state_test  = 87  # =args.random_state_test
                                              )
del windows_dataset

In [7]:
available_classes

array([0, 1, 2, 3, 4])

In [13]:
# Extract weights to balance the loss function
y_true_train = train_set.get_metadata()['target'].to_numpy()
train_weights = torch.Tensor(compute_class_weight('balanced', classes=available_classes, y=y_true_train)).to(device)

### Create model